In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Reads csv and remove some data
df = pd.read_csv('diabetes.csv')

mean_bp = df[df['BloodPressure'] != 0]['BloodPressure'].mean(skipna=True,)
mean_bp = round(mean_bp)

df['BloodPressure'] = df['BloodPressure'].replace(0, mean_bp)

mean_in = df[df['Insulin'] != 0]['Insulin'].mean(skipna=True)
mean_in = round(mean_in)

df['Insulin'] = df['Insulin'].replace(0, mean_in)

df = df.drop(columns=['SkinThickness'])
df

ValueError: Need to specify at least one of 'labels', 'index' or 'columns'

In [56]:
# Split data in X and Y to separate into Features and Outcome. Than split each again in train and test.
X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializes the RandomForest (I have no idea what parameters work best or what arguments to change/tune,
# there are a billion of them and i have no idea what to do, but i guess that's the exercise.)
# Estimators and random_state is suggested by gpt 
clf = RandomForestClassifier(n_estimators=200, random_state=42)

# Fits  evaluating the Model
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.77
